In [1]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/7

In [4]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)

import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [5]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [6]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [45]:
##########################################
# --- Agente 1: Mentor QA --- #
##########################################
def agente_mentor_qa(topico):

    mentor_qa = Agent(
        name="agente_mentor_qa",
        model="gemini-2.0-flash",
        instruction="""
        Você é um profissional especialista em qualidade de software e testes, com a capacidade
        de mentorar quaisquer pessoas, tanto as que atuam nessa área (novatos ou veteranos) quanto àquelas que estão em transição.

        Seus conhecimentos são baseados:

        - Na literatura dos autores de maior relevância na área de Qualidade de Software e Testes como,
        Roger Pressman, Robert C. Martin (Uncle Bob), Martin Fowler, Kent Beck, Elisabeth Hendrickson, Joseph Juran,
        Philip Crosby, Edwards Deming, Armand Feigenbaum, Kaoru Ishikawa e Ian Sommerville.

        - Em conteúdos dos materiais do ISTQB/BSTQB (Syllabus) para os exames de certificação como,
        CTFL, CTAL e demais certificações relevantes para área de qualidade de software e testes.

        - Em documentações oficiais de ferramentas utilizadas na realização dos testes, por exemplo, ferramentas de automação web (Cypress, Selenium, Playwright, Robotframework, etc),
        ferramentas de teste de API (Postman, Insomnia, etc), ferramentas para gestão dos testes e dos bugs (Jira, TestLink, Bugzilla, Qase, etc), ferramentas de testes de performance (Jmeter, K6, Locust, etc), etc.

        Adote a ferramenta do Google (google_search) a seu favor para ter acesso à informações relevantes sobre a área de qualidade de software e testes para te ajudar a engrandecer
        sua base de conhecimento com informações valiosas e úteis.

        Caso o usuário realize perguntas quem são totalmente fora do contexto do qual você é especialista, você tem a liberdade de interromper o assunto de forma totalmente cortês pois, seu foco não é esse, visto que você
        é um especialista em qualidade de software e testes. Busque orientar o usuário sobre seu propósito para que ele evite questionar sobre questões fora do seu contexto.

        """,
        description="Agente que ajuda em qualquer assunto dentro da área de QA",
        tools=[google_search]
    )

    entrada_do_mentor_qa = f"Tópico: {topico}"

    mentoria = call_agent(mentor_qa, entrada_do_mentor_qa)
    return mentoria

In [46]:
#####################################################
# --- Agente 2: Especialista em Plano de Testes --- #
#####################################################
def agente_expert_plano_de_testes(contexto):
  expert_plano_de_testes = Agent(
        name="agente_expert_plano_de_testes",
        model="gemini-2.0-flash",
        instruction="""
        Você é um expert em QA que sabe bastante sobre como elaborar e estruturar planos de testes impecáveis para qualquer projeto de software.

        Seja didático na sua explicação do plano de testes, principalmente para aqueles usuários que você notar que não apresentam tanta familiaridade na
        elaboração de um plano de testes.

        Você pode fornecer um modelo exemplo de um plano de testes baseado na proposta do software que está sendo avaliado para testes.

        Se o usuário te fornecer um exemplo de plano de testes, avalie com atenção, forneça feedbacks construtivos e se for cabível, forneça dicas para melhorias.
        """,
        description="Agente que ajuda com Plano de Testes",
        tools=[google_search]
  )
  entrada_do_expert_plano_de_testes = f"Contexto: {contexto}"
  plano = call_agent(expert_plano_de_testes, entrada_do_expert_plano_de_testes)
  return plano

In [47]:
print("#################################################")
print("######## 🚀 Iniciando o Mestre QA... 🚀 ########")
print("#################################################")
print("\n")
print("🤖 Olá, humano(a)! Eu meu chamo OneQway, sou seu mestre digital de QA.\n")
print("🤖 Para te guiar da melhor forma possível, listo as seguintes opções:")
print("1️⃣ para aprender sobre qualquer assunto dentro da área de QA.")
print("2️⃣ para conversar especificamente da elaboração de um Plano de Testes.")
print("3️⃣ para conversar especificamente da elaboração dos Cenários e Casos de Teste.")
print("4️⃣ para conversar especificamente da elaboração do Relátorio de Bugs.")
print("5️⃣ para conversar especificamente da elobração do Resultado dos Testes.")

opcao_valida = False

while not opcao_valida:  # Enquanto a opção NÃO for válida...
    opcao = input("\n🤖 Agora me forneça uma opção válida: ")

    # MENTOR QA ################################################
    if opcao == "1":
      opcao_valida = True  # Agora a opção é válida, podemos sair do while
      print("\n🚀 Legal! Vamos aprender sobre QA!")
      topico = input("\n🤖 O que você gostaria de aprender hoje dentro do universo QA❓\n")

      if not topico:
          print("\n🤖 Preciso que me informe o que você gostaria de aprender hoje❗\n")
          print("#################################################")
      else:
          print(f"\n🤖 Maravilha! Um momento, estou verificando minha base de conhecimento sobre: {topico}.\n")
          base_de_conhecimento = agente_mentor_qa(topico)
          print("📝 Resultado Obtido da Base de Conhecimento QA")
          display(to_markdown(base_de_conhecimento))
          print("\n🤖 Caso deseje encerrar nossa conversa, basta digitar 'FIM'.\n")

          while True:
            topico = input("🤖 O que gostaria de aprender agora❓\n")
            if topico.lower() == 'fim':
              print('\n🤖 Até a próxima, humano(a)❗')
              break
            else:
              base_de_conhecimento = agente_mentor_qa(topico)
              display(to_markdown(base_de_conhecimento))
              print("\n🤖 Caso deseje encerrar nossa conversa, basta digitar 'FIM'.\n")

    # PLANO DE TESTES ################################################
    elif opcao == "2":
      print("\n📝 Ótimo! Vamos falar sobre Plano de Testes.")
      opcao_valida = True
      contexto = input("\n🤖 O que você gostaria de saber sobre Plano de Testes❓\n")

      if not contexto:
          print("\n🤖 Preciso que me informe o que você gostaria de saber sobre Plano de Testes❗\n")
          print("#################################################")

      else:
          print(f"\n🤖 Maravilha! Um momento, estou analisando seu contexto: {contexto}.\n")
          contexto_plano = agente_expert_plano_de_testes(contexto)
          print("📝 Resultado Obtido ")
          display(to_markdown(contexto_plano))
          print("\n🤖 Caso deseje encerrar nossa conversa, basta digitar 'FIM'.\n")

          while True:
            contexto = input("🤖 O que gostaria de saber agora sobre Plano de Testes❓\n")
            if contexto.lower() == 'fim':
              print('\n🤖 Até a próxima, humano(a)❗')
              break
            else:
              contexto_plano = agente_expert_plano_de_testes(contexto)
              display(to_markdown(contexto_plano))
              print("\n🤖 Caso deseje encerrar nossa conversa, basta digitar 'FIM'.\n")




    # elif opcao == "3":
    #   print("\n✍️ Ótimo! Vamos falar sobre Cenários e Casos de Teste.")
    #   opcao_valida = True

    # elif opcao == "4":
    #   print("\n🐞 Ótimo! Vamos falar sobre Relátorio de Bugs.")
    #   opcao_valida = True

    # elif opcao == "5":
    #   print("\n📊 Ótimo! Vamos falar sobre Resultado dos Testes.")
    #   opcao_valida = True

    else:
      print("\n🤔 Hmm, essa não é uma opção válida. Tente de novo!")


#################################################
######## 🚀 Iniciando o Mestre QA... 🚀 ########
#################################################


🤖 Olá, humano(a)! Eu meu chamo OneQway, sou seu mestre digital de QA.

🤖 Para te guiar da melhor forma possível, listo as seguintes opções:
1️⃣ para aprender sobre qualquer assunto dentro da área de QA.
2️⃣ para conversar especificamente da elaboração de um Plano de Testes.
3️⃣ para conversar especificamente da elaboração dos Cenários e Casos de Teste.
4️⃣ para conversar especificamente da elaboração do Relátorio de Bugs.
5️⃣ para conversar especificamente da elobração do Resultado dos Testes.

🤖 Agora me forneça uma opção válida: 2

📝 Ótimo! Vamos falar sobre Plano de Testes.

🤖 O que você gostaria de saber sobre Plano de Testes❓
Estou com um sistema chamado BugBank, app de um banco digital, que tem as funcionalidades de Login, Cadastro, Transferência e Extrato disponíveis e gostaria de elabora um plano de testes.

🤖 Maravilha! Um momento

> Olá! Elaborar um plano de testes bem estruturado é crucial para garantir a qualidade do BugBank. Vou te ajudar a criar um plano de testes abrangente, adequado às funcionalidades que você mencionou: Login, Cadastro, Transferência e Extrato.
> 
> Primeiramente, vamos entender o que é um plano de testes e como ele se estrutura.
> 
> **O que é um Plano de Testes?**
> 
> Um plano de testes é um documento que descreve o escopo, abordagem, recursos e cronograma das atividades de teste. Ele serve como um guia para a equipe de testes, garantindo que todos estejam alinhados e que todos os aspectos críticos do sistema sejam adequadamente testados.
> 
> **Estrutura de um Plano de Testes:**
> 
> 1.  **Introdução:**
> 
>     *   Objetivo do documento
>     *   Escopo do teste
>     *   Público-alvo
> 2.  **Itens a serem testados:**
> 
>     *   Funcionalidades específicas (Login, Cadastro, Transferência, Extrato)
>     *   Interfaces
>     *   Banco de dados
>     *   Documentação
>     *   Desempenho
>     *   Segurança
> 3.  **Funcionalidades a serem testadas:**
> 
>     *   Descrição detalhada das funcionalidades que serão testadas.
> 4.  **Funcionalidades não testadas:**
> 
>     *   Justificativa para não testar determinadas funcionalidades.
> 5.  **Abordagem de Teste:**
> 
>     *   Tipos de teste (funcional, desempenho, segurança, usabilidade)
>     *   Níveis de teste (unitário, integração, sistema, aceitação)
>     *   Estratégias de teste (testes baseados em risco, testes exploratórios)
> 6.  **Critérios de entrada e saída:**
> 
>     *   Condições que devem ser atendidas para iniciar o teste (ambiente preparado, dados de teste prontos).
>     *   Critérios para considerar o teste concluído (cobertura de testes, taxa de defeitos aceitável).
> 7.  **Entregáveis:**
> 
>     *   Documentos que serão produzidos durante o processo de teste (casos de teste, relatórios de defeitos, relatórios de progresso).
> 8.  **Recursos:**
> 
>     *   Equipe de teste
>     *   Ambientes de teste
>     *   Ferramentas de teste
> 9.  **Cronograma:**
> 
>     *   Prazos para cada atividade de teste
>     *   Marcos importantes
> 10. **Riscos e Contingências:**
> 
>     *   Riscos que podem afetar o processo de teste (atrasos, indisponibilidade de recursos)
>     *   Planos de contingência para mitigar esses riscos
> 
> ### Modelo de Plano de Testes para o BugBank:
> 
> **1. Introdução**
> 
> *   **Objetivo:** Garantir a qualidade e a estabilidade do sistema BugBank, validando as funcionalidades de Login, Cadastro, Transferência e Extrato.
> *   **Escopo:** Este plano abrange os testes funcionais, de desempenho, de segurança e de usabilidade das funcionalidades mencionadas.
> *   **Público-alvo:** Gerentes de projeto, equipe de desenvolvimento, equipe de testes e stakeholders.
> 
> **2. Itens a Serem Testados**
> 
> *   **Funcionalidades:** Login, Cadastro, Transferência, Extrato.
> *   **Interfaces:** Telas do aplicativo, APIs.
> *   **Banco de dados:** Validação da integridade e consistência dos dados.
> *   **Documentação:** Manuais do usuário, documentação técnica.
> 
> **3. Funcionalidades a Serem Testadas**
> 
> *   **Login:**
>     *   **Descrição:** Verificar o acesso de usuários existentes com credenciais válidas e inválidas.
>     *   **Casos de Teste:**
>         *   Login com usuário e senha corretos.
>         *   Login com senha incorreta.
>         *   Login com usuário inexistente.
>         *   Bloqueio de conta após várias tentativas de login inválidas.
> *   **Cadastro:**
>     *   **Descrição:** Validar a criação de novas contas de usuário.
>     *   **Casos de Teste:**
>         *   Cadastro com informações válidas.
>         *   Cadastro com informações inválidas (campos vazios, formatos incorretos).
>         *   Cadastro com CPF/e-mail já existentes.
> *   **Transferência:**
>     *   **Descrição:** Testar a transferência de valores entre contas.
>     *   **Casos de Teste:**
>         *   Transferência para contas válidas.
>         *   Transferência para contas inválidas.
>         *   Transferência com saldo insuficiente.
>         *   Transferência de valores acima do limite permitido.
> *   **Extrato:**
>     *   **Descrição:** Verificar a exibição correta das movimentações financeiras.
>     *   **Casos de Teste:**
>         *   Visualização do extrato com lançamentos recentes.
>         *   Visualização do extrato em diferentes períodos.
>         *   Filtro de lançamentos por tipo (entrada, saída).
> 
> **4. Funcionalidades Não Testadas**
> 
> *   **Notificações Push:** Inicialmente, os testes de notificações push não serão priorizados, mas serão considerados em fases futuras.
>     *   **Justificativa:** Foco nas funcionalidades principais para o lançamento inicial.
> 
> **5. Abordagem de Teste**
> 
> *   **Tipos de Teste:**
>     *   **Funcional:** Validação das funcionalidades conforme especificado.
>     *   **Desempenho:** Testes de carga para verificar a capacidade do sistema em lidar com múltiplos usuários simultâneos.
>     *   **Segurança:** Testes de vulnerabilidade para identificar e corrigir falhas de segurança.
>     *   **Usabilidade:** Avaliação da facilidade de uso do aplicativo.
> *   **Níveis de Teste:**
>     *   **Unitário:** Teste de componentes individuais do sistema.
>     *   **Integração:** Teste da interação entre diferentes componentes.
>     *   **Sistema:** Teste do sistema como um todo.
> *   **Estratégias de Teste:**
>     *   **Testes Baseados em Risco:** Priorização de testes com base nos riscos identificados.
>     *   **Testes Exploratórios:** Testes manuais para descobrir comportamentos inesperados.
> 
> **6. Critérios de Entrada e Saída**
> 
> *   **Critérios de Entrada:**
>     *   Ambiente de teste configurado e disponível.
>     *   Dados de teste preparados.
>     *   Documentação de requisitos revisada e aprovada.
> *   **Critérios de Saída:**
>     *   Todos os casos de teste executados.
>     *   Taxa de defeitos abaixo do limite aceitável (ex: 5%).
>     *   Relatório de testes finalizado e aprovado.
> 
> **7. Entregáveis**
> 
> *   Casos de Teste
> *   Relatórios de Defeitos
> *   Relatórios de Progresso
> *   Relatório de Testes Final
> 
> **8. Recursos**
> 
> *   **Equipe de Teste:**
>     *   Analistas de Testes
>     *   Engenheiros de Automação
> *   **Ambientes de Teste:**
>     *   Ambiente de Desenvolvimento
>     *   Ambiente de Teste (QA)
>     *   Ambiente de Homologação
> *   **Ferramentas de Teste:**
>     *   Selenium (Automação)
>     *   JMeter (Desempenho)
>     *   SonarQube (Análise de Código)
> 
> **9. Cronograma**
> 
> | Atividade             | Prazo      | Responsável        |
> | ---------------------- | ---------- | ------------------ |
> | Planejamento           | 1 semana   | Analista de Testes |
> | Criação de Casos      | 2 semanas  | Analista de Testes |
> | Execução dos Testes   | 3 semanas  | Equipe de Testes   |
> | Relatório Final        | 1 semana   | Analista de Testes |
> 
> **10. Riscos e Contingências**
> 
> *   **Riscos:**
>     *   Atraso na entrega de novas versões do sistema.
>     *   Indisponibilidade do ambiente de teste.
>     *   Defeitos críticos não identificados durante os testes.
> *   **Contingências:**
>     *   Alocação de recursos extras para testes.
>     *   Utilização de ambientes de teste alternativos.
>     *   Revisão e priorização de casos de teste.
> 
> ### Próximos Passos:
> 
> 1.  **Detalhar os Casos de Teste:** Para cada funcionalidade, crie casos de teste específicos, detalhando os passos, dados de entrada e resultados esperados.
> 2.  **Definir a Matriz de Rastreabilidade:** Associe cada caso de teste aos requisitos correspondentes para garantir a cobertura completa.
> 3.  **Implementar a Automação:** Automatize os testes sempre que possível para aumentar a eficiência e a cobertura.
> 4.  **Realizar Testes de Desempenho e Segurança:** Utilize ferramentas específicas para identificar gargalos e vulnerabilidades.
> 5.  **Documentar Tudo:** Mantenha a documentação atualizada e acessível para todos os membros da equipe.
> 
> Este modelo de plano de testes deve te dar um ótimo ponto de partida! Se precisar de ajuda para detalhar algum ponto específico ou tiver dúvidas, é só me perguntar.



🤖 Caso deseje encerrar nossa conversa, basta digitar 'FIM'.

🤖 O que gostaria de saber agora sobre Plano de Testes❓
fim

🤖 Até a próxima, humano(a)❗
